# Quantum Error Correction for Dummies (QEC4D)
# Chapter II - The 3-qbit phase-flip repetition code

In this chapter, we look at how to modify the bit-flip repetition code seen in chapter I to tackle phase-flip errors.

## 1) Overview
The principle is the same as for the X repetition code but this time the second assumption changes to tackle only phase-flip errors instead of the bit-flip ones we previously saw:
- Only a single error can occur in each code word.
- The error type can only be an ***Z-error aka a phase-flip*** (no X-errors).

The goals remain the same.
1. Identify whether an error has occurred (or not, we could get lucky).
2. If an error occurred, identify which physical qbit was affected (this is done by studying the ***error syndrome***, more on this later).
3. If an error occurred, correct it.

In [9]:
# First, a few imports
from copy import deepcopy
from numpy.random import rand, seed
from qiskit import *
from qiskit import Aer
from typing import Tuple

## 2) Building our circuit

### 2-A) Similarities with the X-error repetition code
The encoding step is the same as for the XRC.

In [3]:
def encode_1_into_3(circuit:QuantumCircuit) -> QuantumCircuit:
    """Encode 1 logical qbit (at index 0) into 3 physical qubits."""
    circuit.cnot(0,1)
    circuit.cnot(0,2)
    return circuit

So are the simulation step and the counting of the most frequent occurrence.

In [7]:
def simulate_measurements(circuit:QuantumCircuit, nb_shots:int=1024) -> dict:
    """Simulates measurement results."""
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = backend_sim.run(transpile(circuit, backend_sim), shots=nb_shots)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(circuit)
    return counts

def most_common_output(counts:dict) -> str:
    """ Provides the (q/c)-bit 0>n encoding of the most measured output."""
    return max(counts, key=counts.get)[::-1]

### 2-B) Differences with the X-error repetition code

In [5]:
def hadamard_wall(circuit:QuantumCircuit, start_qb_idx:int, stop_qb_idx:int) -> QuantumCircuit:
    c = deepcopy(circuit)
    for i in range(start_qb_idx, stop_qb_idx):
        c.h(i)
    return c

def toffoli_Z_repetition_code(logical_qb_val:int=0, with_error:bool=False, qb_idx:int=-1) -> QuantumCircuit:
    # Encode the logical qubit information into three physical qubits
    qr1 = QuantumRegister(3)
    circuit = QuantumCircuit(qr1)

    if logical_qb_val == 1:
        circuit.x(0)
        circuit.barrier(qr1) #for visualisation only

    circuit = encode_1_into_3(circuit)
    circuit.barrier(qr1) #for visualisation only

    # Hadamard transformation applied to every qbit
    circuit = hadamard_wall(circuit=circuit, start_qb_idx=0, stop_qb_idx=3)
    circuit.barrier(qr1) #for visualisation only

    # If there is an error, introduce it here on deisred qbit
    if with_error:
        circuit.z(qb_idx)
        circuit.barrier(qr1) #for visualisation only

    # Hadamard transformation applied to every qbit
    circuit = hadamard_wall(circuit=circuit, start_qb_idx=0, stop_qb_idx=3)
    circuit.barrier(qr1) #for visualisation only

    # This step is necessary for the Toffoli-based RC and replaces MBQEC's parity check
    circuit.cnot(0,1)
    circuit.cnot(0,2)
    circuit.barrier(qr1) #for visualisation only
   
    # Add a Toffoli gate to correct potential errors
    circuit.ccx(2,1,0) # the Toffoli gate is also called CCNOT or CCX, they're all one and the same

    return circuit


def measure_toffoli_RC_circuit(circuit:QuantumCircuit, nb_shots:int=1024) -> Tuple[QuantumCircuit, dict]:
    c = deepcopy(circuit)
    
    cr = ClassicalRegister(1) # need one classical register to receive the measurement outcomes
    c.add_register(cr)
    
    c.measure(0,0)

    counts = simulate_measurements(c, nb_shots)

    return c, counts


In [6]:
c = toffoli_Z_repetition_code(logical_qb_val=0)
c.draw('text')

░ ┌───┐ ░ ┌───┐ ░            ░ ┌───┐
q1_0: ──■────■───░─┤ H ├─░─┤ H ├─░───■────■───░─┤ X ├
      ┌─┴─┐  │   ░ ├───┤ ░ ├───┤ ░ ┌─┴─┐  │   ░ └─┬─┘
q1_1: ┤ X ├──┼───░─┤ H ├─░─┤ H ├─░─┤ X ├──┼───░───■──
      └───┘┌─┴─┐ ░ ├───┤ ░ ├───┤ ░ └───┘┌─┴─┐ ░   │  
q1_2: ─────┤ X ├─░─┤ H ├─░─┤ H ├─░──────┤ X ├─░───■──
           └───┘ ░ └───┘ ░ └───┘ ░      └───┘ ░

In [8]:
values_of_logical_qb = [0,1]
idx_of_error = [0,1,2]

example_circ = None
for log_val in values_of_logical_qb:
    for e_idx in idx_of_error:
        circuit = toffoli_Z_repetition_code(logical_qb_val=log_val, with_error=True, qb_idx=e_idx)
        circuit, measurement_res = measure_toffoli_RC_circuit(circuit)
        res = most_common_output(measurement_res)
        print(f"For a phase-flip error on qbit index {e_idx} on a circuit encoding {log_val} the corrected circuit yielded {res}")
        example_circ = circuit

example_circ.draw('text')

For a bitflip error on qbit index 0 on a circuit encoding 0 the corrected circuit yielded 0
For a bitflip error on qbit index 1 on a circuit encoding 0 the corrected circuit yielded 0
For a bitflip error on qbit index 2 on a circuit encoding 0 the corrected circuit yielded 0
For a bitflip error on qbit index 0 on a circuit encoding 1 the corrected circuit yielded 1
For a bitflip error on qbit index 1 on a circuit encoding 1 the corrected circuit yielded 1
For a bitflip error on qbit index 2 on a circuit encoding 1 the corrected circuit yielded 1


┌───┐ ░            ░ ┌───┐ ░       ░ ┌───┐ ░            ░ ┌───┐┌─┐
q39_0: ┤ X ├─░───■────■───░─┤ H ├─░───────░─┤ H ├─░───■────■───░─┤ X ├┤M├
       └───┘ ░ ┌─┴─┐  │   ░ ├───┤ ░       ░ ├───┤ ░ ┌─┴─┐  │   ░ └─┬─┘└╥┘
q39_1: ──────░─┤ X ├──┼───░─┤ H ├─░───────░─┤ H ├─░─┤ X ├──┼───░───■───╫─
             ░ └───┘┌─┴─┐ ░ ├───┤ ░ ┌───┐ ░ ├───┤ ░ └───┘┌─┴─┐ ░   │   ║ 
q39_2: ──────░──────┤ X ├─░─┤ H ├─░─┤ Z ├─░─┤ H ├─░──────┤ X ├─░───■───╫─
             ░      └───┘ ░ └───┘ ░ └───┘ ░ └───┘ ░      └───┘ ░       ║ 
 c5: 1/════════════════════════════════════════════════════════════════╩═
                                                                       0

### 2-C) Why it works
So it works. That's undeniable. But how? We barely changed a thing from the XRC, intuitively one might expect to design a wholly new different code from scratch and apply it, but that's simply not the case.

To understand why this work we need to look more closely at what X and Z errors actually are.

#### 2-C-I) Pauli X and Z
Fundamentally, what we call a bit-flip error or X-error is simply the application of a Pauli X matrix to the state of the system. Ie:
- $\mathbf{X}\ket{0} \mapsto \ket{1}$
- $\mathbf{X}\ket{1} \mapsto \ket{0}$
- $\mathbf{X}\ket{+} \mapsto \ket{+}$ behaves like $\mathbb{I}$
- $\mathbf{X}\ket{-} \mapsto \ket{-}$ behaves like $\mathbb{I}$

And as for the phase-flip or Z-error, it again is simply the application of a Pauli Z matrix:
- $\mathbf{Z}\ket{0} \mapsto \ket{0}$ behaves like $\mathbb{I}$
- $\mathbf{Z}\ket{1} \mapsto -\ket{1}$ flips the sign but otherwise behaves like $\mathbb{I}$
- $\mathbf{Z}\ket{+} \mapsto \ket{-}$
- $\mathbf{Z}\ket{-} \mapsto \ket{+}$

If we agree for a moment to disregard the change of sign in $\mathbf{Z}\ket{1}$, what we have is schematically:
- $\mathbf{X}$ flips $\ket{0}, \ket{1}$ to their opposite and acts as $\mathbb{I}$ on $\ket{+}, \ket{-}$.
- $\mathbf{Z}$ flips $\ket{+}, \ket{-}$ to their opposite and acts as $\mathbb{I}$ on $\ket{1}, \ket{0}$.
Pauli $\mathbf{X}$ and $\mathbf{Z}$ are basically the same operator but in a different basis. They do the *same* thing.

To convince ourselves of this identity, let's run a few example circuits and compare their results.

In [12]:
c1 = QuantumCircuit(QuantumRegister(1), ClassicalRegister(1))
c2 = deepcopy(c1)

c1.x(0) # apply a bit-flip on qbit 0
c1.measure(0,0)
cpt1 = simulate_measurements(c1, 1024)
most_freq_1 = most_common_output(cpt1)

c2.h(0) # apply a Hadamard
c2.z(0) # then a phase-flip
c2.h(0) # then another Hadamard, all to qbit 0
c2.measure(0,0)
cpt_2 = simulate_measurements(c2, 1024)
most_freq_2 = most_common_output(cpt_2)

print(f"X|0> = {most_freq_1}")
print(f"HZH|0> = {most_freq_2}")

X|0> = 1
HZH|0> = 1


#### 2-C-II) X and Z basis and measurements

When we talk about $\ket{0}, \ket{1}$ we are referring to quantum states represented in the Z-basis.
Now when we talk about $\ket{+}, \ket{-}$, we are referring to quantum states represented in the X-basis.

Let's remember that not only can we write $\ket{+} = \frac{\ket{0} + \ket{1}}{\sqrt{2}}$ and $\ket{-} = \frac{\ket{0} - \ket{1}}{\sqrt{2}}$ but we also have $\ket{0} = \frac{\ket{+} + \ket{-}}{\sqrt{2}}$ and $\ket{1} = \frac{\ket{+} - \ket{-}}{\sqrt{2}}$.

And changing between both is done simply by applying a Hadamard gate $\mathbf{H}$ such that:
- $\mathbf{H}\ket{0} \mapsto \ket{+}$
- $\mathbf{H}\ket{1} \mapsto \ket{-}$
- $\mathbf{H}\ket{+} \mapsto \ket{0}$
- $\mathbf{H}\ket{-} \mapsto \ket{1}$

#### 2-C-III) Circling back
Now why did we discuss all this in the first place? To understand why the circuit to correct phase-flips was almost the same as the one to correct bit-flips with only a bunch of $\mathbf{H}$ gates in the middle.
This is ultimately because, as we've shown empirically earlier, $\mathbf{H}\mathbf{Z}\mathbf{H} = \mathbf{X}$.

So in the case where we *know* we're interested in phase-flips (rather than bit-flips), the logic goes as follows:
- We start off in the computational (Z) basis with qbits in $\ket{0}, \ket{1}$ states.
- We first apply $\mathbf{H}$ to change into the X basis. Now our qbits are in $\ket{+}, \ket{-}$ states.
- We let phase-flip error happen, this will turn a $\ket{+}$ into a $\ket{-}$ and vice-versa.
- We apply another  $\mathbf{H}$ to change back to the Z basis and get back $\ket{0}, \ket{1}$ states, here any $\ket{+}$ is mapped to a $\ket{0}$ and any $\ket{-}$ to a $\ket{1}$ so the former phase-flip errors are transformed into bit-flip ones.
- We correct for the bit-flip errors using the familiar XRC.

##### Why isn't it th other way around?
Aha, excellent question! Why don't we construct a circuit to correct for phase-flip errors in the X-basis first and then manipulate it with $\mathbf{H}$ to shift to the bit-flip error handling in Z-basis?
Two reasons for this. The first boring one is that by convention, most circuits (and libraries representing them) assume that things starts off in the Z basis. We can always change to the X basis but that requires applying $\mathbf{H}$-transforms on every qbit.


Try to be comfortable with these properties of Pauli X and Z and X and Z basis as they are key features in QEC that will keep showing up.